# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [2]:
# import libraries
import pandas as pd
import os
from sqlalchemy import create_engine

In [3]:
# load messages dataset
messages = pd.read_csv('data\disaster_messages.csv')
# Remove duplicates
messages = messages[~messages.duplicated(subset='id')]
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
# load categories dataset
categories = pd.read_csv('data\disaster_categories.csv')
# Remove duplicates
categories = categories[~categories.duplicated(subset='id')]

# Split categories into 36 separate columns
cat_split = categories['categories'].str.split(';', expand=True)
# Rename the categories columns; use the first row to infer the names
cat_split.columns = cat_split.iloc[0,:].apply(lambda x : x[:-2])
# Convert the entries to 0/1
cat_split = cat_split.applymap(lambda x : x[-1]).astype('int')
# Replace erroneous values
cat_split.replace({2:0}, inplace=True)
# Bring back the 'id' column (will be joined on index)
categories = cat_split.join(categories['id'])
categories.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report', 'id'],
      dtype='object')

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
# merge datasets
df = pd.merge(messages, categories, on='id', how='outer', validate='1:1')
print(messages.shape, categories.shape, df.shape)

(26180, 4) (26180, 37) (26180, 40)


In [6]:
# Take a closer look at the labels

# import seaborn as sn
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(13, 10))
# sn.heatmap(categories.drop(columns='id').corr(), ax=fig.gca(), cmap='rocket_r')
# fig.show()


df.query("aid_related == 0")[['medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid']].sum(axis=1).value_counts()

0    15339
dtype: int64

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates


In [ ]:
# drop duplicates


In [ ]:
# check number of duplicates


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [52]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('merged', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [3]:
def load_data(messages_filepath, categories_filepath):
    """Load the datasets containing messages and categories.

    Args:
        messages_filepath: str
            Path to a .csv file with the messages dataset.
        categories_filepath: str
            Path to a .csv file with the corresponding categories.

    Returns:
        A combined dataframe.
    """

    # load messages dataset
    messages = pd.read_csv(messages_filepath)

    # load categories dataset
    categories = pd.read_csv(categories_filepath)

    # Merge datasets
    df = pd.merge(messages, categories, on='id', how='outer')
    
    return df


def clean_data(df):
    """Cleans the megred dataframe of messages and categories."""

    # Remove duplicates
    df = df[~df.duplicated(subset='id')]

    # Split categories into 36 separate columns
    cat_split = df['categories'].str.split(';', expand=True)
    # Rename the categories columns; use the first row to infer the names
    cat_split.columns = cat_split.iloc[0,:].apply(lambda x : x[:-2])
    # Convert the entries to 0/1
    cat_split = cat_split.applymap(lambda x : x[-1]).astype('int')
    # Replace erroneous values ('2' corresponds to the 'No' answer)
    cat_split.replace({2:0}, inplace=True)
    # Combine with the rest of the dataframe
    df = df.loc[:, df.columns != 'categories'].join(cat_split)
    # Remove uninformative columns
    df = df.loc[:, df.columns != 'child_alone']
    
    return df

def save_data(df, database_filename):
    """Save the cleaned dataset to a SQLite database.

    Args:
        df: pandas.DataFrame
            Cleaned dataset; will be saved as a table called 'merged'.
        database_filename: str
            Filename of the database including the '.db' file extension.

    Returns:
        None
    """

    # Remove the file if it already exists
    if os.path.exists(database_filename):
        os.remove(database_filename)

    engine = create_engine('sqlite:///{}'.format(database_filename))
    df.to_sql('merged', engine, index=False)


    
df = load_data('data\disaster_messages.csv', 'data\disaster_categories.csv')

df = clean_data(df)

save_data(df, 'DisasterResponse.db')